<a href="https://colab.research.google.com/github/shreyagiri98/Sampling_Assignment/blob/main/SamplingAssignment_Shreya_102303684.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install imbalanced-learn xgboost lightgbm


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN, SMOTETomek


In [3]:
data = pd.read_csv("/content/Creditcard_data.csv")
X = data.drop("Class", axis=1)
y = data["Class"]

scaler = StandardScaler()
X = scaler.fit_transform(X)


In [12]:
samplers = {
    "SMOTE": SMOTE(),
    "RandomOver": RandomOverSampler(),
    "RandomUnder": RandomUnderSampler(),
    "SMOTEENN": SMOTEENN(),
    "SMOTETomek": SMOTETomek()
}


In [13]:
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "SVC": SVC(),
    "XGBClassifier": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}


In [14]:
results = pd.DataFrame(index=models.keys(), columns=samplers.keys())

for s_name, sampler in samplers.items():
    X_res, y_res = sampler.fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

    for m_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        results.loc[m_name, s_name] = round(acc*100, 2)

results


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:48:04] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:48:04] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:48:04] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:48:05] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:48:06] WARNING: /w

,SMOTE,RandomOver,RandomUnder,SMOTEENN,SMOTETomek
LogisticRegression,93.14,93.14,25.0,93.52,93.14
DecisionTreeClassifier,97.39,99.35,75.0,98.29,96.73
RandomForestClassifier,99.35,100.0,0.0,100.0,99.35
SVC,96.73,96.08,0.0,98.98,96.73
XGBClassifier,98.69,100.0,0.0,99.66,99.02


In [16]:
best_sampling = results.idxmax(axis=1)
best_accuracy = results.max(axis=1)

final_result = pd.DataFrame({
    "BST": best_sampling,
    "Accuracy": best_accuracy
})

final_result


,BST,Accuracy
LogisticRegression,SMOTEENN,93.52
DecisionTreeClassifier,RandomOver,99.35
RandomForestClassifier,RandomOver,100.0
SVC,SMOTEENN,98.98
XGBClassifier,RandomOver,100.0
